In [ ]:
'''
Author : Rupesh Garsondiya
github : @Rupeshgarsondiya
Topic  : Transfer Learning In CNN Using PyTorch
'''


'\nAuthor : Rupesh Garsondiya\ngithub : @Rupeshgarsondiya\nTopic  : Transfer Learning In CNN Using PyTorch\n'

# **Transfer Learning**

**Reference :**PyTroch playlist By Nitish Sir (Campus X)

**About This notebook :** You can gain practical knowledge on how to apply transfer learning in convolutional neural network using Torch.

**prerequisites :**Understanding of CNN, Basic of torch, Transfer Learning

**Suggestion :**If you want to understand how transfer learning in convolutional neural networks work, check out my deep learning repository. In that repo and check the file i give a file name **Trasfer_Learning_In_CNN-.ipynb** , I explain all deep learning concepts in detail with both theoretical and practical guides. Check it out!

**Let's now focus on PyTorch**

In [ ]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


!kaggle datasets download -d zalando-research/fashionmnist

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/zalando-research/fashionmnist
License(s): other
 96% 66.0M/68.8M [00:00<00:00, 255MB/s]
100% 68.8M/68.8M [00:00<00:00, 230MB/s]


In [ ]:
import zipfile
zipfile = zipfile.ZipFile('/content/fashionmnist.zip')
zipfile.extractall('/content')
zipfile.close()

In [ ]:
import os
import time
import torch
import numpy as np
import torchvision
import torch.nn as nn
from PIL import Image
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from torchvision import datasets, models, transforms
from tempfile import TemporaryDirectory

cudnn.benchmark = True
plt.ion()   # interactive mode


In [ ]:
import pandas as pd

df_train = pd.read_csv('/content/fashion-mnist_train.csv')
df_train.head()

df_test = pd.read_csv('/content/fashion-mnist_test.csv')
df_test.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Extract features and labels for training set
# X_train: All columns except 'label' from the df_train DataFrame are used as features for the training set
# y_train: The 'label' column from df_train DataFrame is used as the target labels for the training set
X_train = df_train.drop('label', axis=1).values
y_train = df_train['label'].values

# Extract features and labels for test set
# X_test: All columns except 'label' from the df_test DataFrame are used as features for the test set
# y_test: The 'label' column from df_test DataFrame is used as the target labels for the test set
X_test = df_test.drop('label', axis=1).values
y_test = df_test['label'].values

# Print the shape (dimensions) of the training and testing data
# This helps confirm the structure and size of the datasets
print('X_train : ', X_train.shape)  # Prints the shape of the training features
print('y_train : ', y_train.shape)  # Prints the shape of the training labels
print('X_test : ', X_test.shape)    # Prints the shape of the test features
print('y_test : ', y_test.shape)    # Prints the shape of the test labels

X_train :  (60000, 784)
y_train :  (60000,)
X_test :  (10000, 784)
y_test :  (10000,)


# **Define the Transformation**

Because in torchvision, they define what type of input is accepted by VGG16 or other models, you just need to check it out.

**Supported input types and conventions :**
Most transformations accept both PIL images and tensor inputs. Both CPU and CUDA tensors are supported. The result of both backends (PIL or Tensors) should be very close. In general, we recommend relying on the tensor backend for performance. The conversion transforms may be used to convert to and from PIL images, or for converting dtypes and ranges.

Tensor image are expected to be of shape (C, H, W), where C is the number of channels, and H and W refer to height and width. Most transforms support batched tensor input. A batch of Tensor images is a tensor of shape (N, C, H, W), where N is a number of images in the batch. The v2 transforms generally accept an arbitrary number of leading dimensions (..., C, H, W) and can handle batched images or batched videos.

**Reference :**https://pytorch.org/vision/main/transforms.html


In [ ]:
# Define Transformation
from torchvision.transforms import transforms

custom_transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.CenterCrop((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std =[0.229, 0.224, 0.225])
])

# **Create CustomDataset class**

In [ ]:
from PIL import Image
import numpy as np

class CustomDataset(torch.utils.data.Dataset):
  def __init__(self,features,labels,transform):

    self.features = features
    self.labels = labels
    self.transforms = transform

  def __len__(self):

    return len(self.features)

  def __getitem__(self,index):

    # resize images
    image = self.features[index].reshape(28,28)

    # convert datatype np.uint8
    image = image.astype(np.uint8)

    # chnage Gray Scale image to Colour image (RGB)
    image = np.stack((image,)*3,axis=-1)  # set chennels like stack axis = -1 ===> (H,W,C) with out this parametr or default value ====> (C,H,W) --> Dimension Of the image

    # convert array to PIL images
    image = Image.fromarray(image)

    # applay transofrmation
    image = self.transforms(image)

    # return
    return image,torch.tensor(self.labels[index])

In [ ]:
from torch.utils.data import DataLoader,Dataset
# create train dataset object
train_dataset = CustomDataset(X_train,y_train,transform=custom_transform)


# create test dataset object
test_dataset = CustomDataset(X_test,y_test,custom_transform)

train_loader = DataLoader(train_dataset,batch_size=8,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=8,shuffle=False)

# **VGG16 Architechture**


In [ ]:
# fetch pre-trained model
'''
1. Import the VGG16 model
2. Freeze or No update in Feature part weight
3. Create our own classifire part
'''

import torchvision.models as models


vgg16 = models.vgg16(pretrained=True)
print('='*20,'VGG16 Feature Part','='*20)
print(vgg16.features)
print('='*20,'VGG16 Classifier Part','='*20)
print(vgg16.classifier)

# Freeze the Weight
for param in vgg16.features.parameters():  # we fetch all the parameter for the feature part and set required_gread = Fale  it mean during training no gradient update
    param.requires_grad = False


# Chnage the classifire part of the VGG16
vgg16.classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=1024, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=1024, out_features=512, bias=True),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=512, out_features=10, bias=True)
)

print('='*20,'Own Classifier Part','='*20)
print(vgg16.classifier)


# Model move on GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16 = vgg16.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


==================== VGG16 Feature Part ====================
Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, di

In [ ]:
learning_rate = 0.01
epochs = 3

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier.parameters(),lr=learning_rate) # optimizer only perform on the classifire part of the architechture

# **Define Training Loop**

In [ ]:
# Training Loop

# Iterate through the specified number of epochs
for epochs in range(epochs):

    # Initialize variable to track total loss for this epoch
    total_epochs_loss = 0

    # Iterate through each batch of data in the train_loader (this handles batching and shuffling of the data)
    for batch_feature, label in train_loader:

        batch_feature = batch_feature.to(device)
        label = label.to(device)

        # Forward pass: Pass the batch of features through the model to get the predictions (output)
        output = vgg16(batch_feature)

        # Calculate the loss between the predicted output and true labels
        loss = criterion(output, label)

        # Add the current loss to the total loss for the epoch
        total_epochs_loss += loss.item()

        # Backward pass: Compute gradients of the loss with respect to model parameters
        optimizer.zero_grad()  # Clear the previous gradients
        loss.backward()        # Backpropagate the loss to compute new gradients

        # Update model parameters (weights and biases) using the optimizer
        optimizer.step()

    # Calculate the average loss for the epoch (divide by the number of batches in the train_loader)
    avg_loss = total_epochs_loss / len(train_loader)

    # Print the average loss for the current epoch
    print(f'Epoch : {epochs+1} Loss : {avg_loss}')


Epoch : 1 Loss : 2.5410263346791266
Epoch : 2 Loss : 2.6241194193045296
Epoch : 3 Loss : 2.718522989622752


# **No changes in the Inference code**

In [1]:
# set model to eval mode
# we need to define our model explicitly you use as evalulate beacuse in deep leaarning some time behave diffrent during training and testing
# like dropout we fropout apply on only during traing we not apply dropout on testing
# same in case of the batch normalization

vgg16.eval()

# Evaluation code: Evaluates the model on the test set

# Initialize variables to track the total number of samples and number of correct predictions
total = 0
correct = 0

# Disable gradient calculation to save memory and computation during evaluation
# `torch.no_grad()` ensures that gradients are not calculated during this phase
with torch.no_grad():

  # Iterate through each batch in the test_loader (this handles batching of test data)
  for batch_feature, label in test_loader:

    batch_feature = batch_feature.to(device)
    label = label.to(device)

    # Forward pass: Pass the batch of test features through the model to get the predictions (output)
    output = vgg16(batch_feature)

    # Get the predicted class by selecting the index with the highest output score for each sample
    # `torch.max(output, 1)` returns the maximum value and its index along dimension 1 (the classes)
    _, predicted = torch.max(output, 1)

    # Update the total number of samples
    total = total + label.size(0)

    # Update the number of correct predictions by comparing predicted classes with actual labels
    correct = correct + (predicted == label).sum().item()

# Calculate and print the accuracy as the ratio of correct predictions to the total number of samples
print(f'Accuracy : {correct/total}')

NameError: name 'vgg16' is not defined